In [1]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [2]:
# data = pd.read_excel('serieA-2023-24.xlsx')
# data1920 = pd.read_excel('data/engdata/19-20.xlsx', index_col='Date')
# data2021 = pd.read_excel('data/engdata/20-21.xlsx', index_col='Date')
# data2122 = pd.read_excel('data/engdata/21-22.xlsx', index_col='Date')
# data2223 = pd.read_excel('data/engdata/22-23.xlsx', index_col='Date')
# data2324 = pd.read_excel('data/engdata/23-24.xlsx', index_col='Date')

data1920 = pd.read_excel('data/engdata/Serie B/19-20-E.xlsx', index_col='Date')
data2021 = pd.read_excel('data/engdata/Serie B/20-21-E.xlsx', index_col='Date')
data2122 = pd.read_excel('data/engdata/Serie B/21-22-E.xlsx', index_col='Date')
data2223 = pd.read_excel('data/engdata/Serie B/22-23-E.xlsx', index_col='Date')
data2324 = pd.read_excel('data/engdata/Serie B/23-24-E.xlsx', index_col='Date')

data = data1920

data = pd.concat([data, data2021, data2122, data2223, data2324])
print(data)
# data['Cluster'] = np.where(data['FTR'] == 'H', 1, np.where(data['FTR'] == 'A', 2, 0))

data.tail()

                        Div      Time     HomeTeam   AwayTeam  FTHG  FTAG FTR  \
Date                                                                            
1970-01-01 01:00:43.700  I2  20:00:00         Pisa  Benevento     0     0   D   
1970-01-01 01:00:43.701  I2  17:00:00      Crotone    Cosenza     0     0   D   
1970-01-01 01:00:43.701  I2  17:00:00  Salernitana    Pescara     3     1   H   
1970-01-01 01:00:43.701  I2  20:00:00       Ascoli    Trapani     3     1   H   
1970-01-01 01:00:43.701  I2  20:00:00   Cittadella     Spezia     0     3   A   
...                      ..       ...          ...        ...   ...   ...  ..   
1970-01-01 01:00:45.422  I2  19:30:00  FeralpiSalo    Ternana     0     1   A   
1970-01-01 01:00:45.422  I2  19:30:00        Lecco     Modena     2     3   A   
1970-01-01 01:00:45.422  I2  19:30:00     Reggiana      Parma     1     1   D   
1970-01-01 01:00:45.422  I2  19:30:00       Spezia    Venezia     2     1   H   
1970-01-01 01:00:45.422  I2 

,Div,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,MaxCAHA,AvgCAHH,AvgCAHA,HomeGoalsCumulative,AwayGoalsCumulative,HomePointsCumulative,AwayPointsCumulative,HomeGoalsConcededCumulative,AwayGoalsConcededCumulative,MatchGoal
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-01 01:00:45.422,I2,19:30:00,FeralpiSalo,Ternana,0,1,A,0.0,1.0,A,...,2.02,1.91,1.91,44,42,33,40,64,50,1
1970-01-01 01:00:45.422,I2,19:30:00,Lecco,Modena,2,3,A,1.0,3.0,A,...,2.03,1.86,1.94,33,38,26,44,71,45,5
1970-01-01 01:00:45.422,I2,19:30:00,Reggiana,Parma,1,1,D,1.0,0.0,H,...,1.90,2.07,1.76,37,65,46,75,44,34,2
1970-01-01 01:00:45.422,I2,19:30:00,Spezia,Venezia,2,1,H,0.0,1.0,A,...,1.95,1.94,1.87,34,68,41,70,48,44,3
1970-01-01 01:00:45.422,I2,19:30:00,Sudtirol,Palermo,0,1,A,0.0,0.0,D,...,2.08,1.83,1.98,46,61,47,53,47,53,1


In [3]:
resultFileName = 'data/mergedDataSerieB.xlsx'
data.to_excel(resultFileName)